## Data Collector For Other Work For Bert Training

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Bert/all_data_for_bert_training.csv"
)

df = df[["initial_prompt", "ability"]]

In [ ]:
ct_df1 = pd.read_csv(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/reference_code/cs4_benchmark/CS4_dataset/Instruction-based Constraints.csv"
)
print(ct_df1.columns)
ct_df1["initial_prompt"] = ct_df1["Instruction "] + "\n" + ct_df1["Constraints"]

ct_df1["ability"] = "CT"
ct_df1 = ct_df1[["initial_prompt", "ability"]]

Index(['Instruction Number', 'Instruction ', 'Number of Constraints',
       'Constraints'],
      dtype='object')


In [4]:
ct_df2 = pd.read_csv(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/reference_code/cs4_benchmark/CS4_dataset/Story-based Constraints.csv"
)

ct_df2["initial_prompt"] = ct_df2["Instruction "] + "\n" + ct_df2["Constraints"]

ct_df2["ability"] = "CT"
ct_df2 = ct_df2[["initial_prompt", "ability"]]

In [5]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["keyword"],
    template=(
        "I'll be submitting your next responses to a 'Good Scientific Idea' expert review panel. If "
        "they consider your idea to be a good one, you'll receive a reward. Your assigned keyword "
        "is: '{keyword}'. You may provide background information. The idea MUST be within 100 words "
        "(including background information). (Note: good scientific ideas should be novel, verifiable, "
        "practically valuable, and able to advance the field.)."
    ),
)

ct_df3 = pd.read_csv(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/reference_code/liveideabench/liveideabench_hf.csv"
)[0:1000]

ct_df3["initial_prompt"] = ct_df3["keywords"].apply(
    lambda x: prompt_template.format(keyword=x)
)

ct_df3["ability"] = "CT"
ct_df3 = ct_df3[["initial_prompt", "ability"]]

In [6]:
main_df = pd.concat([df, ct_df1, ct_df2, ct_df3])

In [ ]:
main_df.ability.value_counts()

ability
CR     24732
IF     24426
SUM    14798
MT     14699
ICL     3920
CT      2088
Name: count, dtype: int64

In [8]:
ct_df3.shape

(1000, 2)

In [9]:
ICL_df = pd.read_csv(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Results/ICL/exp_result_Meta-Llama-3-8B-Instruct_20240531032532_951068_evaluated.csv"
)

In [10]:
ICL_df = (
    ICL_df[ICL_df["dataset"] == "triviaqa_e"]
    .drop_duplicates(subset=["initial_prompt"])
    .reset_index(drop=True)
)

In [11]:
ICL_df["ability"] = "ICL"
ICL_df = ICL_df[["initial_prompt", "ability"]]

In [12]:
new_df = pd.concat([main_df, ICL_df])

In [13]:
new_df.ability.value_counts()

ability
CR     24732
IF     24426
SUM    14798
MT     14699
ICL     3985
CT      2088
Name: count, dtype: int64

In [14]:
new_df = new_df.drop_duplicates(subset=["initial_prompt"]).reset_index(drop=True)

In [15]:
new_df.ability.value_counts()

ability
MT     600
CT     556
IF     501
ICL    145
SUM    109
CR      50
Name: count, dtype: int64

In [16]:
new_df.columns

Index(['initial_prompt', 'ability'], dtype='object')

In [17]:
CRASS_FTM_main_data_set_df = pd.read_csv(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Source/CR/CRASS-data-set/CRASS_FTM_main_data_set.csv",
    encoding="latin1",
    sep=";",
)

CRASS_FTM_main_data_set_df["ability"] = "CR"

In [18]:
arraycounter = 0
system_message = "You are a conscientious assistant, and you must answer my questions exactly as my questions require."
instruction = "Please reference the following three examples (including premise, query, possible answers, and correct answer) to answer the final query."
initial_prompt = """
Instruction: {system_message}
Input: 
{INSTRUCTION}
{INPUT}
"""
mixtral_instruct_initial_prompt = """<s>[INST]
{system_message}
{INSTRUCTION}
{INPUT}
[/INST] 
</s>"""

llama2_chat_initial_prompt = """
<s>[INST] <<SYS>>
{system_message}
<</SYS>>
{INSTRUCTION}
{INPUT}
[/INST]
"""

llama3_chat_init_prompt = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system_message}
<|eot_id|><|start_header_id|>user<|end_header_id|>
{INSTRUCTION}
{INPUT}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""


few_shot_prompt = """Premise: A feather falls from a skyscraper. 
Query: What would have happened if a computer had fallen from the skyscraper? 
Possible answers: The computer would have remained intact. That is not possible. The computer would have been crushed. 
Correct answer: The computer would have been crushed.

Premise: A lightning hits a tree. 
Query: What would have happened if a marble would have hit the tree? 
Possible answers: It would have burned down. Nothing special would have happened. The tree would have kissed the lightning. 
Correct answer: Nothing special would have happened. 

Premise: A man drinks a beer. 
Query: What would have happened if the man had drunk a rainbow? 
Possible answers: It would have been tasty. It would have been awful. That is not possible. 
Correct answer: That is not possible.

"""

for _, row in CRASS_FTM_main_data_set_df.iterrows():
    instruction = "Please reference the following three examples (including premise, query, possible answers, and correct answer) to answer the final query."
    few_shot_prompt = """
    Examples: 

    Premise: {}
    Query: {}
    Possible answers: {}
    Correct answer: {}
    """.format(
        row.Premise, row.QCC, row.CorrectAnswer, row.Answer1
    )

    hf_inputs = "Premise: {}\nQuery: {}".format(row.Premise, row.QCC)
    hf_cl1 = row.CorrectAnswer
    hf_cl2 = row.Answer1
    hf_cl3 = row.Answer2
    prompt = (
        few_shot_prompt
        + hf_inputs
        + "\nPossible answers: "
        + hf_cl1
        + hf_cl2
        + hf_cl3
        + "\nCorrect answer:"
    )
    # Add instruction and prompt to dataframe
    CRASS_FTM_main_data_set_df.at[_, "instruction"] = instruction
    CRASS_FTM_main_data_set_df.at[_, "input"] = prompt


from langchain import PromptTemplate

initial_prompt_template = PromptTemplate.from_template(initial_prompt)
llama2_chat_initial_prompt_template = PromptTemplate.from_template(
    llama2_chat_initial_prompt
)
mixtral_instruct_initial_prompt_template = PromptTemplate.from_template(
    mixtral_instruct_initial_prompt
)
llama3_chat_initial_prompt_template = PromptTemplate.from_template(
    llama3_chat_init_prompt
)


def generate_initial_prompts(row):
    input = row["input"]
    instruction = row["instruction"]
    initial_prompt = initial_prompt_template.format(
        INSTRUCTION=instruction, INPUT=input, system_message=system_message
    )

    llama2_chat_initial_prompt = llama2_chat_initial_prompt_template.format(
        INSTRUCTION=instruction, INPUT=input, system_message=system_message
    )

    llama3_chat_initial_prompt = llama3_chat_initial_prompt_template.format(
        INSTRUCTION=instruction, INPUT=input, system_message=system_message
    )

    mixtral_instruct_initial_prompt = mixtral_instruct_initial_prompt_template.format(
        INSTRUCTION=instruction, INPUT=input, system_message=system_message
    )

    row["initial_prompt"] = initial_prompt
    row["llama2_chat_initial_prompt"] = llama2_chat_initial_prompt
    row["mixtral_instruct_initial_prompt"] = mixtral_instruct_initial_prompt
    row["llama3_chat_initial_prompt"] = llama3_chat_initial_prompt
    row["INSTRUCTION"] = input
    row["INPUT"] = instruction
    return row


import numpy as np

CRASS_FTM_main_data_set_df = CRASS_FTM_main_data_set_df.apply(
    generate_initial_prompts, axis=1
)
temperatures = np.arange(0.1, 2.0, 0.3)
temperature_df = pd.DataFrame({"Temperature": temperatures})
merged_df = temperature_df.merge(CRASS_FTM_main_data_set_df, how="cross")

In [19]:
CR_df = CRASS_FTM_main_data_set_df[["initial_prompt", "ability"]]

In [20]:
new_df = pd.concat([CR_df, new_df])

In [21]:
new_df.ability.value_counts()

ability
MT     600
CT     556
IF     501
CR     324
ICL    145
SUM    109
Name: count, dtype: int64

#### Data Augmentation

In [22]:
new_df.to_csv(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Bert/all_data_for_bert_training_non_repetitive.csv",
    index=False,
)

In [23]:
import random
from nltk.corpus import wordnet


def synonym_replacement(sentence, n=1):
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    random.shuffle(random_word_list)

    for random_word in random_word_list[:n]:
        synonyms = wordnet.synsets(random_word)
        if synonyms:
            synonym = synonyms[0].lemmas()[0].name()
            new_words = [synonym if word == random_word else word for word in new_words]
    return " ".join(new_words)


def random_insertion(sentence, n=1):
    words = sentence.split()
    for _ in range(n):
        random_word = random.choice(words)
        synonyms = wordnet.synsets(random_word)
        if synonyms:
            synonym = synonyms[0].lemmas()[0].name()
            insert_position = random.randint(0, len(words))
            words.insert(insert_position, synonym)
    return " ".join(words)


def random_swap(sentence, n=1):
    words = sentence.split()
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return " ".join(words)


def random_deletion(sentence, p=0.2):
    words = sentence.split()
    if len(words) == 1:  # 如果只有一个单词，直接返回
        return sentence
    new_words = [word for word in words if random.uniform(0, 1) > p]
    return " ".join(new_words) if new_words else random.choice(words)


def spelling_error_injection(sentence, n=1):
    def typo(word):
        if len(word) <= 1:
            return word
        idx = random.randint(0, len(word) - 2)
        return word[:idx] + word[idx + 1] + word[idx] + word[idx + 2 :]

    words = sentence.split()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        words[idx] = typo(words[idx])
    return " ".join(words)


keyboard_map = {
    "a": ["q", "w", "s", "z"],
    "b": ["v", "g", "h", "n"],
}


def keyboard_error_injection(sentence, n=1):
    def keyboard_typo(word):
        chars = list(word)
        idx = random.randint(0, len(chars) - 1)
        if chars[idx] in keyboard_map:
            chars[idx] = random.choice(keyboard_map[chars[idx]])
        return "".join(chars)

    words = sentence.split()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        words[idx] = keyboard_typo(words[idx])
    return " ".join(words)


from collections import Counter

# 假设已有训练语料库的unigram分布
unigram_freq = Counter({"the": 5000, "a": 3000, "cat": 1000})


def unigram_noising(sentence, n=1):
    words = sentence.split()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        replacement_word = random.choices(
            list(unigram_freq.keys()), weights=list(unigram_freq.values()), k=1
        )[0]
        words[idx] = replacement_word
    return " ".join(words)

In [24]:
import pandas as pd
import nltk

nltk.download("wordnet")


# 定义增强函数集合|
def augment_text(text, times=5):
    augmented_texts = []

    for _ in range(times):
        # EDA 操作
        augmented_texts.append(synonym_replacement(text))
        augmented_texts.append(random_insertion(text))
        augmented_texts.append(random_swap(text))
        augmented_texts.append(random_deletion(text))

        # 噪声注入操作
        augmented_texts.append(spelling_error_injection(text))
        augmented_texts.append(keyboard_error_injection(text))
        augmented_texts.append(unigram_noising(text))

    return augmented_texts


augmented_data = []
for index, row in new_df.iterrows():
    original_prompt = row["initial_prompt"]
    augmented_prompts = augment_text(original_prompt, 2)
    for augmented_prompt in augmented_prompts:
        augmented_data.append(
            {
                "initial_prompt": original_prompt,
                "augmented_prompt": augmented_prompt,
                "ability": row["ability"],
            }
        )

augmented_df = pd.DataFrame(augmented_data)

[nltk_data] Downloading package wordnet to /home/snt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
augmented_df.ability.value_counts()

ability
MT     8400
CT     7784
IF     7014
CR     4536
ICL    2030
SUM    1526
Name: count, dtype: int64

In [26]:
augmented_df = augmented_df.drop_duplicates(subset=["augmented_prompt"]).reset_index(
    drop=True
)

In [27]:
augmented_df.ability.value_counts()

ability
MT     6662
CT     6187
IF     5576
CR     3617
ICL    1605
SUM    1233
Name: count, dtype: int64

In [28]:
augmented_df.to_csv(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Bert/all_data_for_bert_training_augmented.csv",
    index=False,
)

In [29]:
import pandas as pd

test = pd.read_csv(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Bert/all_data_for_bert_training_augmented.csv"
)